# 📌 Eindopdracht: Bouw een complete ETL-pipeline voor declaraties
## 🔎 Case: Automatiseren van declaratieverwerking voor Menzis
**Doel:** Bouw een **volledige ETL-pipeline** die declaraties **dagelijks verwerkt, opschoont en veilig opslaat in Delta Lake**.

## 1️⃣ Extract: Laad data uit verschillende bronnen

In [ ]:
from pyspark.sql import SparkSession

# Start een Spark sessie
spark = SparkSession.builder.appName("ETL_Declaraties").getOrCreate()

# Laad declaraties dataset
df_declaraties = spark.read.csv("/mnt/data/declaraties_dataset.csv", header=True, inferSchema=True)
df_verzekerden = spark.read.csv("/mnt/data/verzekerden_dataset.csv", header=True, inferSchema=True)

# Toon de eerste records
df_declaraties.show(5)
df_verzekerden.show(5)

## 2️⃣ Transform: Opschonen en bewerken van data

In [ ]:
from pyspark.sql.functions import col, expr

# Verwijder missende waarden
df_clean = df_declaraties.dropna()

# Filter onrealistische declaraties (bedragen tussen €10 en €500)
df_filtered = df_clean.filter((col("Bedrag") >= 10) & (col("Bedrag") <= 500))

# Mask gevoelige persoonsgegevens
df_masked = df_verzekerden.withColumn("Naam", expr("concat(substr(Naam, 1, 1), '****')"))

# Toon resultaat
df_filtered.show(5)
df_masked.show(5)

## 3️⃣ Load: Opslaan in Delta Lake (Brons, Zilver, Goud)

In [ ]:
# Opslaan in Brons-laag
df_declaraties.write.format("delta").mode("overwrite").save("/mnt/data/brons/declaraties")

# Opslaan in Zilver-laag
df_filtered.write.format("delta").mode("overwrite").save("/mnt/data/zilver/declaraties")

# Opslaan in Goud-laag (geaggregeerde data)
df_goud = df_filtered.groupBy("Zorgtype").sum("Bedrag")
df_goud.write.format("delta").mode("overwrite").save("/mnt/data/goud/declaraties_per_zorgtype")

## 4️⃣ Automatisering: Maak een Databricks Job

In [ ]:
def run_etl():
    # Start de ETL pipeline
    print("ETL Pipeline gestart...")
    df_filtered.write.format("delta").mode("overwrite").save("/mnt/data/zilver/declaraties")
    print("ETL Pipeline voltooid ✅")

# Run de ETL pipeline
run_etl()

## 5️⃣ Beveiliging en Performance-optimalisatie

In [ ]:
# Role-Based Access Control (RBAC)
spark.sql("GRANT SELECT ON delta.`/mnt/data/zilver/declaraties` TO `data_analyst`;")

# Partitioning en Z-Ordering
df_partitioned = df_filtered.repartition("Zorgtype")
df_partitioned.write.format("delta").mode("overwrite").partitionBy("Zorgtype").save("/mnt/data/zilver/declaraties_partitioned")
spark.sql("OPTIMIZE delta.`/mnt/data/zilver/declaraties_partitioned` ZORDER BY (Declaratie_Datum)")

## 6️⃣ Logging en Monitoring

In [ ]:
from pyspark.sql.functions import lit
import datetime

def log_error(error_message):
    error_df = spark.createDataFrame(
        [(str(datetime.datetime.now()), error_message)], 
        ["Timestamp", "Error"]
    )
    error_df.write.format("delta").mode("append").save("/mnt/data/logs/errors")

# Simuleer een fout
log_error("Testfout: Geen toegang tot Delta Lake tabel")

## 🎯 Bonus opdrachten
1. **Exporteer de Goud-laag naar een Parquet-bestand en upload dit naar Azure Data Lake.**
2. **Gebruik Delta Lake Time Travel om oude versies van declaraties te bewaren.**
3. **Voeg een retry-mechanisme toe aan de job, zodat deze bij een fout automatisch opnieuw start.**